# Basic OLS


## Loading Packages

In [1]:
using StatsBase, Distributions
using Compat, Missings        #in Julia 0.6 
#using Dates, DelimitedFiles  #in Julia 0.7

include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")

NWFn

## Loading Data

In [2]:
x  = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100) 
x = nothing                   

println(size(Rme))

(388,)


## Point Estimates

Consider the linear regression

$
y_{t}=\beta^{\prime}x_{t}+\varepsilon_{t},
$

where $y_{t}$ is a scalar and $x_{t}$ is $k\times1$. The OLS estimate is

$
\hat{\beta} = S_{xx}^{-1}S_{xy}, \: \text{ where } \: 
S_{xx}      = \frac{1}{T}\sum\nolimits_{t=1}^{T}x_{t}x_{t}^{\prime}
\: \text{ and } \:
S_{xy}      = \frac{1}{T}\sum\nolimits_{t=1}^{T}x_{t}y_{t}.
$

(The $1/T$ terms clearly cancel, but are sometimes useful to keep to preserve
numerical precision.)

Instead of these sums (loops over $t$), matrix multiplication can be used to
speed up the calculations. Create matrices $X_{T\times k}$ and $Y_{T\times1}$
by letting $x_{t}^{\prime}$ and $y_{t}$ be the $t^{th}$ rows

$
X_{T\times k}=\left[
\begin{array}[c]{l}
x_{1}^{\prime}\\
\vdots\\
x_{T}^{\prime}
\end{array}
\right] \ \text{ and } \ Y_{T\times1}=\left[
\begin{array}[c]{l}
y_{1}\\
\vdots\\
y_{T}
\end{array}
\right].
$

We can then calculate the same matrices as

$
S_{xx}       =X^{\prime}X/T \ \text{ and } \: S_{xy}=X^{\prime}Y/T \: \text{, so } \: 
\hat{\beta}  =(X^{\prime}X)^{-1}X^{\prime}Y.
$

However, instead of inverting $S_{xx}$, we typically get much better numerical
precision by solving the system of $T$ equations

$
X_{T\times k}b_{k\times1}=Y_{T\times1}
$

for the vector $b$ that minimizes the sum of squared errors. This
is easily done by using the command
```
b = X\Y
```

In [3]:
println("Three different ways to calculate OLS estimates")

Y = Rme
T = size(Y,1)
X = [ones(T) RSMB RHML]

K = size(X,2)
S_xx = zeros(K,K)
S_xy = zeros(K,1)
for t = 1:T
    local x_t, y_t
    global S_xx, S_xy
    x_t = X[t,:]               #a vector
    y_t = Y[t:t,:]             
    S_xx = S_xx + x_t*x_t'/T   #KxK
    S_xy = S_xy + x_t*y_t/T    #Kx1
end
b1 = inv(S_xx)*S_xy          #OLS coeffs, version 1

b2 = inv(X'X)*X'Y            #OLS coeffs, version 2

b3 = X\Y                     #OLS coeffs, version 3

println("\nb1, b2 and b3")
printmat([b1 b2 b3])

Three different ways to calculate OLS estimates

b1, b2 and b3
     0.007     0.007     0.007
     0.217     0.217     0.217
    -0.429    -0.429    -0.429



## Distribution of OLS Estimates

The distribution of the estimates is (typically)

$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V)
\: \text{ where } \: V=S_{xx}^{-1} S S_{xx}^{-1}
$

where $S$ is the covariance matrix of $\sqrt{T}\bar{g}$, where $\bar{g}$
is the sample average of

$
g_{t}=x_{t}(y_{t}-x_{t}^{\prime}\beta),
$

and $S_{xx}$ is defined as 

$
S_{xx}=-\sum_{t=1}^{T}x_{t}x_{t}^{\prime}/T.
$

In [4]:
b = X\Y
u = Y - X*b              #residuals
g = X.*u                 #TxK, moment conditions

println("\navg moment conditions")
printmat(Compat.mean(g,dims=1))       #0.7 syntax

S   = NWFn(g,1)               #Newey-West covariance matrix
Sxx = -X'X/T
V   = inv(Sxx)'S*inv(Sxx)     #Cov(sqrt(T)*b)

println("\nb and std(b)")
printmat([b3 sqrt.(diag(V/T))])


avg moment conditions
     0.000    -0.000     0.000


b and std(b)
     0.007     0.002
     0.217     0.124
    -0.429     0.108



## A Function for OLS

In [5]:
"""
    OlsFn(y,x,m=1)

LS of y on x; for one dependent variable

# Usage
(b,res,yhat,V,R2a) = OlsFn(y,x,m)

# Input
- `y::Array`:     Tx1, the dependent variable
- `x::Array`:     Txk matrix of regressors (including deterministic ones)
- `m::Int`:       scalar, bandwidth in Newey-West  

# Output
- `b::Array`:     kx1, regression coefficients
- `u::Array`:     Tx1, residuals y - yhat
- `yhat::Array`:  Tx1, fitted values x*b
- `V::Array`:     kxk matrix, covariance matrix of sqrt(T)b
- `R2a::Number`:  scalar, R2 value

"""
function OlsFn(y,x,m=0)
    T    = size(y,1)
    b    = x\y
    yhat = x*b
    u    = y - yhat
    g    = x.*u   
    S0   = NWFn(g,m)            #Newey-West covariance matrix
    Sxx  = -x'x/T
    V    = inv(Sxx)'S0*inv(Sxx)  
    R2a  = 1 - var(u)/var(y)
    return b,u,yhat,V,R2a
end

OlsFn

In [6]:
(b4,_,_,V,R2a) = OlsFn(Y,X,1)
println("\n with NW standard errors")
printmat([b4 sqrt.(diag(V/T))])


 with NW standard errors
     0.007     0.002
     0.217     0.124
    -0.429     0.108



## Testing a Hypothesis

Since the estimator $\hat{\beta}_{_{k\times1}}$ satisfies

$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V_{k\times k})  ,
$

we can easily apply various tests. To test a joint linear hypothesis of the
form

$
\gamma_{q\times1}=R\beta-a,
$

use the test

$
(R\beta-a)^{\prime}(\Lambda/T)  ^{-1}(R\beta
-a)\overset{d}{\rightarrow}\chi_{q}^{2} \: \text{, where } \: \Lambda=RVR^{\prime}.
$

In [7]:
R = [0 1 0;               #testing if b(2)=0 and b(3)=0
     0 0 1]
a = [0;0]
Γ = R*V*R'
test_stat = (R*b-a)'inv(Γ/T)*(R*b-a)
println("\ntest-statictic and 10% critical value of chi-square(2)")
printmat([test_stat 4.61])


test-statictic and 10% critical value of chi-square(2)
    26.059     4.610



## Regression Diagnostics: Testing All Slope Coefficients

The function in the next cell tests all slope coefficients of the regression.

In [8]:
"""
    OlsR2TestFn(R2a,T,k)

"""
function OlsR2TestFn(R2a,T,k)

  RegrStat = T*R2a/(1-R2a)
  pval     = 1 - cdf(Chisq(k-1),RegrStat)
  Regr     = [RegrStat pval (k-1)]

  return Regr

end

OlsR2TestFn

In [9]:
Regr = OlsR2TestFn(R2a,T,size(X,2))

println("Test of all slopes: stat, p-val, df")
printmat(Regr)

Test of all slopes: stat, p-val, df
    60.165     0.000     2.000



## Regression Diagnostics: Autocorrelation of the Residuals

The function in the next cell estimates autocorrelations, calculates the DW and Box-Pierce statistics.

In [10]:
"""
    OlsAutoCorrFn(u,m=1)

Test the autocorrelation of OLS residuals

# Input:
- `u::Array`:   Tx1, residuals
- `m::Int`:     scalar, number of lags in autocorrelation and Box-Pierce test

# Output
- `AutoCorr::Array`:    mx2, autorrelation and p-value
- `DW::Number`:         scalar, DW statistic
- `BoxPierce::Array`:   1x2, Box-Pierce statistic and p-value

# Requires
- StatsBase, Distributions

"""
function OlsAutoCorrFn(u,m=1)

  T = size(u,1)

  Stdu = std(u)
  rho  = autocor(u,1:m)

  pval      = 2*(1.0 .- cdf.(Normal(0,1),sqrt(T)*abs.(rho)))
  AutoCorr  = [rho pval]

  BPStat    = T*sum(rho.^2)
  pval      = 1 - cdf(Chisq(m),BPStat)
  BoxPierce = [BPStat pval]

  dwStat    = mean(diff(u).^2)/Stdu^2

  return AutoCorr,dwStat,BoxPierce

end

OlsAutoCorrFn

In [11]:
(AutoCorr,dwStat,BoxPierce) = OlsAutoCorrFn(u,3)

println("    lag        autoCorr  p-val:")
printmat([1:3 AutoCorr])

printlnPs("DW:",dwStat)

println("\nBoxPierce: stat, p-val")
printmat(BoxPierce)

    lag        autoCorr  p-val:
     1.000     0.074     0.142
     2.000    -0.037     0.464
     3.000     0.019     0.706

DW:     1.849

BoxPierce: stat, p-val
     2.831     0.418



## Regression Diagnostics: Heteroskedasticity

The function in the next cell performs White's test for heteroskedasticity.

In [12]:
"""
    OlsWhitesTestFn(u,x)

# Input:
- `u::Array`:   Tx1, residuals
- `x::Array`:   Txk, regressors

"""
function OlsWhitesTestFn(u,x)

  (T,k) = (size(x,1),size(x,2))

  psi = zeros(T,round(Int,k*(k+1)/2))        #matrix of cross products of x
  vv = 0
  for i = 1:k, j = i:k
      vv        = vv + 1  
      psi[:,vv] = x[:,i].*x[:,j]           #all cross products, incl own
  end
    
  (_,_,_,_,R2a) = OlsFn(u.^2,psi)   #White's test for heteroskedasticity
    
  WhiteStat = T*R2a/(1-R2a)
  pval      = 1 - cdf(Chisq(size(psi,2)-1),WhiteStat)
  White     = [WhiteStat pval (size(psi,2)-1)]

  return White
   
end

OlsWhitesTestFn

In [13]:
White = OlsWhitesTestFn(u,X)

println("White: stat,p-val, df ")
printmat(White)

White: stat,p-val, df 
    77.278     0.000     5.000



# A Function for SURE (OLS)


Consider the linear regression

$
y_{it}=\beta_i^{\prime}x_{t}+\varepsilon_{it}, 
$

where $i=1,2,..,n$ indicates $n$ different dependent variables. The regressors are the *same* across the regressions.

In [14]:
"""
    OlsSureFn(y,x,m=1)

LS of y on x; for one n dependent variables, same regressors

# Usage
(b,res,yhat,Covb,R2a) = OlsSureFn(y,x,m)

# Input
- `y::Array`:     Txn, the n dependent variables
- `x::Array`:     Txk matrix of regressors (including deterministic ones)
- `m::Int`:       scalar, bandwidth in Newey-West  

# Output
- `b::Array`:     n*kx1, regression coefficients
- `u::Array`:     Txn, residuals y - yhat
- `yhat::Array`:  Txn, fitted values x*b
- `V::Array`:     matrix, covariance matrix of sqrt(T)vec(b)
- `R2a::Number`:  n vector, R2 value

"""
function OlsSureFn(y,x,m=0)
    (T,n) = (size(y,1),size(y,2))
    k     = size(x,2)
    b     = x\y
    yhat  = x*b
    u     = y - yhat   
    g     = zeros(T,n*k)
    for i = 1:n                      
      vv      = (1+(i-1)*k):(i*k)   #1:k,(1+k):2k,...
      g[:,vv] = x.*u[:,i]           #moment conditions for y[:,i] regression
    end    
    S0    = NWFn(g,m)            #Newey-West covariance matrix
    Sxxi  = -x'x/T 
    Sxx_1 = kron(Matrix(1.0I,n,n),inv(Sxxi))
    V     = Sxx_1 * S0 * Sxx_1
    R2a   = 1.0 .- Compat.var(u,dims=1)./Compat.var(y,dims=1)    #0.7 syntax
    return b,u,yhat,V,R2a
end



OlsSureFn

In [15]:
println("regressing [RSMB RHML] on Rme: [vec(coef) vec(std)]")
(b,u,yhat,V,R2a) = OlsSureFn([RSMB RHML],[ones(T) Rme],1)
printmat([vec(b) sqrt.(diag(V/T))])


R = [1 0 -1 0]                               #Testing if the alphas are the same
Γ = R*V*R'
test_stat = (R*vec(b))'inv(Γ/T)*(R*vec(b))
println("test-statictic of alpha1=alpha2 and 10% critical value of chi-square(1)")
printmat([test_stat 2.71])

regressing [RSMB RHML] on Rme: [vec(coef) vec(std)]
     0.001     0.001
     0.164     0.034
     0.005     0.002
    -0.230     0.051

test-statictic of alpha1=alpha2 and 10% critical value of chi-square(1)
     2.112     2.710

